In [ ]:
# -*- coding: utf-8 -*-
import os
import torch

# --- LangChain 및 관련 라이브러리 임포트 ---
# Core
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain.docstore.document import Document # 직접 Document 객체 생성 시

# Memory
from langchain.memory import ConversationBufferWindowMemory

# Loaders
from langchain_community.document_loaders import TextLoader, DirectoryLoader, WikipediaLoader

# Embeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

# Vectorstores
from langchain_community.vectorstores import Chroma

# LLMs (HuggingFace Pipeline 사용 예시)
from langchain_community.llms import HuggingFacePipeline

# Text Splitters
from langchain.text_splitter import RecursiveCharacterTextSplitter

# --- Hugging Face 및 PyTorch 관련 라이브러리 임포트 ---
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
# 파인튜닝 모델 로딩 (qLoRA)에 필요할 수 있음
try:
    from peft import PeftModel행 전 확인 및 준비사항:

경로 설정: 코드 상단의 FINETUNED_LLM_PATH, BASE_MODEL_NAME_OR_PATH, VECTOR_DB_PATH, WIKI_DATA_PATH, ADVANCED_PROMPTS_PATH 를 실제 환경에 맞게 정확히 수정하세요. 특히 파인튜닝된 어댑터 경로와 베이스 모델 이름이 중요합니다.
라이브러리 설치: 필요한 모든 라이브러리가 설치되어 있는지 확인하세요. 특히 peft 라이브러리는 qLoRA 모델 로딩에 필수적일 수 있습니다.
Bash

pip install langchain langchain-community langchain-huggingface chromadb sentence-transformers torch transformers accelerate bitsandbytes pypdf wikipedia peft # 필요한 라이브러리 설치
RAG 데이터 준비: WIKI_DATA_PATH와 ADVANCED_PROMPTS_PATH에 RAG에 사용할 실제 데이터 파일들을 위치시키세요. (위키 데이터는 예시 키워드 외에 프로젝트에 필요한 내용을 추가하거나, 직접 관련 텍스트 파일을 준비하는 것이 좋습니다.)
GPU 메모리: 파인튜닝된 LLM을 로드하고 실행하기에 충분한 GPU VRAM이 필요합니다. 메모리가 부족하면 로딩에 실패하거나 매우 느릴 수 있습니다. (필요시 BitsAndBytesConfig를 사용한 양자화 설정을 조정하세요.)
실행:

위 코드를 conversational_rag.py 와 같은 이름으로 저장합니다.
터미널에서 실행합니다: python conversational_rag.py
실행되면 설정 정보와 함께 각 단계별 진행 상황이 출력됩니다.
[User Input (Korean)]: 프롬프트가 나타나면 간단한 한국어 프롬프트를 입력하고 엔터를 누릅니다.
시스템이 RAG 검색 및 LLM 추론을 통해 상세한 영어 프롬프트를 생성하여 출력합니다.
이전 대화를 기억하므로, 다음 턴에는 "거기에 파란색 모자를 추가해줘" 와 같이 이전 프롬프트를 수정/보완하는 지시를 내릴 수 있습니다.
exit를 입력하면 프로그램이 종료됩니다.

    print("Warning: 'peft' library not found. pip install peft")
    PeftModel = None # peft 없으면 로딩 불가 처리

# --- 1. 설정 ---
# !!! 중요: 사용자의 환경에 맞게 경로와 모델 이름을 정확히 수정하세요 !!!
FINETUNED_LLM_PATH = "/path/to/your/finetuned-kanan-nano-2.1b-instruct/adapter" # <<< 파인튜닝된 어댑터 가중치 경로
BASE_MODEL_NAME_OR_PATH = "NousResearch/Llama-2-7b-chat-hf" # <<< 파인튜닝 시 사용한 베이스 모델 경로 또는 이름
VECTOR_DB_PATH = "./chroma_db_midjourney_rag_conv" # Chroma DB 저장 경로
EMBEDDING_MODEL_NAME = "intfloat/e5-large-v2" # 임베딩 모델 (다국어 지원)

# RAG 소스 데이터 경로 설정 (예시)
WIKI_DATA_PATH = "./rag_data/wiki"
ADVANCED_PROMPTS_PATH = "./rag_data/advanced_prompts"
os.makedirs(WIKI_DATA_PATH, exist_ok=True)
os.makedirs(ADVANCED_PROMPTS_PATH, exist_ok=True)

# 예시 고급 프롬프트 데이터 (실제 데이터로 교체 필요)
example_adv_prompt = """
masterpiece, best quality, ultra-detailed, illustration, beautiful detailed eyes, 1girl, cat ears, solo, long hair, blonde hair, blue eyes, school uniform, sailor collar, serafuku, pleated skirt, outdoors, sky, clouds, looking at viewer, dynamic angle, depth of field, cinematic lighting, volumetric light, detailed background of a vibrant cityscape at sunset --ar 16:9 --style raw --q 2
"""
with open(os.path.join(ADVANCED_PROMPTS_PATH, "example_prompt.txt"), "w", encoding='utf-8') as f:
    f.write(example_adv_prompt)

# 텍스트 분할 설정
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 100

# 디바이스 설정 (GPU 우선 사용)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 대화 메모리 설정
MEMORY_WINDOW_SIZE = 3 # 최근 몇 턴의 대화를 기억할지 설정

# qLoRA 로딩을 위한 설정 (파인튜닝 시 사용한 설정과 일치시켜야 할 수 있음)
# 예시: 4비트 로딩 설정 (필요시 주석 해제 및 조정, bitsandbytes 필요)
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16 # 또는 torch.float16
# )

print(f"--- Configuration ---")
print(f"Device: {DEVICE}")
print(f"Embedding Model: {EMBEDDING_MODEL_NAME}")
print(f"Base LLM for Finetune: {BASE_MODEL_NAME_OR_PATH}")
print(f"Finetuned Adapter Path: {FINETUNED_LLM_PATH}")
print(f"Vector DB Path: {VECTOR_DB_PATH}")
print(f"Memory Window Size: {MEMORY_WINDOW_SIZE}")
print(f"--------------------")

# --- 2. RAG용 데이터 로드 및 분할 ---
print("\n[Phase 1/7] Loading RAG source documents...")
all_docs = []

# 2-1. 고급 프롬프트 예시 로드
try:
    prompt_loader = DirectoryLoader(ADVANCED_PROMPTS_PATH, glob="**/*.txt", loader_cls=TextLoader, show_progress=True, loader_kwargs={'encoding': 'utf-8'})
    advanced_prompt_docs = prompt_loader.load()
    for doc in advanced_prompt_docs:
        doc.metadata["source"] = "advanced_prompt_example"
    all_docs.extend(advanced_prompt_docs)
    print(f"- Loaded {len(advanced_prompt_docs)} advanced prompt examples.")
except Exception as e:
    print(f"- Error loading advanced prompts: {e}")

# 2-2. 위키피디아 정보 로드 (예시)
wiki_keywords = ["Cat", "Beach", "Forest", "Cyberpunk", "Steampunk", "Oil painting", "Illustration", "Photorealistic"] # 예시 키워드
wiki_docs_to_load = []
print(f"- Loading Wikipedia data for keywords (max 1 doc per keyword): {wiki_keywords}...")
try:
    for keyword in wiki_keywords:
        # WikipediaLoader는 영어 위키 기준, 한국어 필요 시 다른 방식 고려
        loader = WikipediaLoader(query=keyword, load_max_docs=1, doc_content_chars_max=2000) # 내용 길이 제한
        docs = loader.load()
        for doc in docs:
            doc.metadata["source"] = f"wikipedia_{keyword}"
        wiki_docs_to_load.extend(docs)
    all_docs.extend(wiki_docs_to_load)
    print(f"- Loaded {len(wiki_docs_to_load)} documents from Wikipedia.")
except Exception as e:
    print(f"- Error loading Wikipedia data: {e}. Skipping Wikipedia loading.")
    print("- Consider manually adding relevant Wikipedia text files.")

# 2-3. 문서 분할
if not all_docs:
    print("\nError: No documents loaded for RAG. Please check data paths and loaders. Exiting.")
    exit()

print(f"\n- Total documents loaded: {len(all_docs)}")
print("- Splitting documents...")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
texts = text_splitter.split_documents(all_docs)
print(f"- Split into {len(texts)} text chunks.")

# --- 3. 임베딩 모델 로드 ---
print("\n[Phase 2/7] Initializing embedding model...")
try:
    model_kwargs = {'device': DEVICE}
    encode_kwargs = {'normalize_embeddings': True} # E5 모델 권장 사항
    embeddings = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL_NAME,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    print("- Embedding model loaded successfully.")
except Exception as e:
    print(f"\nError initializing embedding model: {e}. Exiting.")
    exit()

# --- 4. 벡터 DB 생성 및 저장 (Chroma) ---
print("\n[Phase 3/7] Initializing Chroma vector store...")
try:
    if os.path.exists(VECTOR_DB_PATH):
        print(f"- Loading existing vector store from {VECTOR_DB_PATH}")
        vectordb = Chroma(persist_directory=VECTOR_DB_PATH, embedding_function=embeddings)
    else:
        print(f"- Creating new vector store at {VECTOR_DB_PATH} (this may take a while)...")
        vectordb = Chroma.from_documents(
            documents=texts,
            embedding=embeddings,
            persist_directory=VECTOR_DB_PATH
        )
        vectordb.persist()
    retriever = vectordb.as_retriever(search_kwargs={"k": 3}) # 관련성 높은 청크 3개 검색
    print("- Vector store initialized and retriever created.")
except Exception as e:
    print(f"\nError initializing vector store: {e}. Exiting.")
    exit()

# --- 5. 파인튜닝된 LLM 로드 ---
# !!! 중요: qLoRA 등 파인튜닝 방식에 맞춰 정확한 로딩 코드 필요 !!!
print(f"\n[Phase 4/7] Initializing Finetuned LLM...")
llm = None
try:
    if not PeftModel:
        raise ImportError("PeftModel is not available. Cannot load LoRA adapter.")

    # 1. 베이스 모델 로드
    print(f"- Loading base model: {BASE_MODEL_NAME_OR_PATH}")
    # qLoRA 사용 시 bitsandbytes 설정 (bnb_config) 필요할 수 있음
    model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_NAME_OR_PATH,
        # quantization_config=bnb_config, # 4비트 로딩 설정 적용 시
        torch_dtype=torch.bfloat16 if DEVICE == "cuda" else torch.float32, # GPU 환경에 맞게 조정
        device_map="auto" # 자동 디바이스 매핑 (GPU 활용)
    )
    print("- Base model loaded.")

    # 2. 어댑터(파인튜닝 가중치) 로드 및 병합 (Peft 사용)
    print(f"- Loading adapter (finetuned weights) from: {FINETUNED_LLM_PATH}")
    model = PeftModel.from_pretrained(model, FINETUNED_LLM_PATH)
    # 필요하다면 모델 병합: model = model.merge_and_unload() (병합 후에는 어댑터 경로 불필요)
    print("- Adapter loaded.")

    # 3. 토크나이저 로드 (베이스 모델의 토크나이저 사용)
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME_OR_PATH)

    # 4. LangChain 파이프라인 생성
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256, # 생성할 최대 토큰 수 (미드저니 프롬프트 길이 고려)
        temperature=0.7,    # 다양성
        top_p=0.95,         # 샘플링 확률
        repetition_penalty=1.1 # 반복 방지
    )
    llm = HuggingFacePipeline(pipeline=pipe)
    print("- Finetuned LLM loaded successfully via HuggingFacePipeline.")

except Exception as e:
    print(f"\n--------------------------------------------------------------")
    print(f"Error loading finetuned LLM: {e}")
    print(f"Please check:")
    print(f"  1. Correctness of paths: '{BASE_MODEL_NAME_OR_PATH}' and '{FINETUNED_LLM_PATH}'")
    print(f"  2. Availability of 'peft' library and its compatibility.")
    print(f"  3. Sufficient GPU memory and required dependencies (transformers, accelerate, bitsandbytes).")
    print(f"Exiting application as LLM is not available.")
    print(f"--------------------------------------------------------------")
    exit()

# --- 6. 메모리 및 RAG 체인 구성 ---
print("\n[Phase 5/7] Configuring Conversational RAG chain...")

# 6-1. 메모리 초기화
try:
    memory = ConversationBufferWindowMemory(
        k=MEMORY_WINDOW_SIZE, # 기억할 턴 수
        memory_key="chat_history", # 프롬프트 템플릿과 일치
        return_messages=True # ChatPromptTemplate과 호환
    )
    print(f"- Conversation memory initialized (Window size: {MEMORY_WINDOW_SIZE})")
except Exception as e:
    print(f"\nError initializing memory: {e}. Exiting.")
    exit()

# 6-2. 프롬프트 템플릿 정의
try:
    conversational_prompt_template = ChatPromptTemplate.from_messages([
        ("system", """You are an expert prompt engineer specializing in Midjourney. Your task is to translate and refine a simple Korean prompt into a highly detailed English prompt, considering the ongoing conversation history and provided context. Base your response primarily on the user's latest instruction, using history and context for enhancement."""),
        MessagesPlaceholder(variable_name="chat_history"), # 대화 기록 삽입 위치
        ("human", """**Context from Knowledge Base & Examples (based on latest instruction):**
{context}

**User's Latest Instruction (Korean):**
{korean_input}

**Instructions:**
1. Review the conversation history (`chat_history`) to understand the progression.
2. Focus on the `User's Latest Instruction (Korean)` for the main goal of this turn.
3. Use the provided `Context` (retrieved based on the latest instruction) to add relevant details, styles, camera angles, lighting etc.
4. Generate an updated or refined English Midjourney prompt based on the latest instruction and context, considering the history.
5. Output ONLY the final English prompt string for this turn, without any extra explanation or conversation.

**Refined English Midjourney Prompt:**
"""),
    ])
    print("- Conversational prompt template created.")
except Exception as e:
    print(f"\nError creating prompt template: {e}. Exiting.")
    exit()

# 6-3. 헬퍼 함수: 검색된 문서 포맷팅
def format_docs(docs):
    formatted_docs = []
    for i, doc in enumerate(docs):
        source = doc.metadata.get('source', 'unknown')
        content_preview = doc.page_content[:200].replace("\n", " ") # 미리보기, 줄바꿈 제거
        formatted_docs.append(f"--- Context Source [{source}] ---\n{content_preview}...")
    return "\n\n".join(formatted_docs) if formatted_docs else "No relevant context found."

# 6-4. 핵심 RAG 체인 정의 (LCEL)
# 메모리 로드/저장은 이 체인 외부 루프에서 처리
try:
    core_rag_chain = (
        RunnableParallel(
            # context는 현재 korean_input 기반으로 검색 후 포맷팅
            context=RunnableLambda(lambda x: retriever.invoke(x["korean_input"])) | format_docs,
            # korean_input과 chat_history는 그대로 전달 (이후 단계에서 추출)
            passthrough=RunnablePassthrough()
        )
        | RunnableLambda(lambda x: { # 프롬프트 템플릿에 필요한 변수만 정확히 매핑
              "context": x["context"],
              "korean_input": x["passthrough"]["korean_input"], # passthrough 딕셔너리에서 값 추출
              "chat_history": x["passthrough"]["chat_history"] # passthrough 딕셔너리에서 값 추출
          })
        | conversational_prompt_template # 프롬프트 적용
        | llm # LLM 호출
        | StrOutputParser() # 출력 파싱 (문자열)
    )
    print("- Conversational RAG chain configured successfully.")
except Exception as e:
    print(f"\nError configuring RAG chain: {e}. Exiting.")
    exit()


# --- 7. 대화형 프롬프트 변환 실행 ---
print("\n[Phase 6/7] Ready for Conversational Prompt Transformation.")
print("Enter your simple Korean prompt. Type 'exit' to quit.")

while True:
    try:
        korean_input = input("\n[User Input (Korean)]: ")
        if korean_input.lower() == 'exit':
            break
        if not korean_input.strip():
            continue

        print("\n[System] Processing...")

        # 1. 메모리에서 이전 대화 기록 로드
        loaded_memory = memory.load_memory_variables({})
        chat_history = loaded_memory.get('chat_history', []) # memory_key와 일치

        # 2. 핵심 RAG 체인 호출 (현재 입력 + 로드된 기록 전달)
        inputs = {"korean_input": korean_input, "chat_history": chat_history}
        english_prompt_output = core_rag_chain.invoke(inputs)

        # 3. 현재 턴의 입력과 출력을 메모리에 저장
        memory.save_context({"input": korean_input}, {"output": english_prompt_output})

        print("\n--- Generated English Midjourney Prompt ---")
        print(english_prompt_output)
        print("-------------------------------------------")

        # (디버깅용) 검색된 컨텍스트 확인
        # retrieved_docs = retriever.invoke(korean_input)
        # print("\n[Debug] Retrieved Context:")
        # print(format_docs(retrieved_docs))
        # print("-------------------------")


    except KeyboardInterrupt:
        print("\n\n[System] Interrupted by user. Exiting.")
        break
    except Exception as e:
        print(f"\n[Error] An error occurred during processing: {e}")
        # 오류 발생 시 다음 루프 계속 진행
        continue

print("\n[Phase 7/7] Exiting application.")

실실실행 전 확인 및 준비사항:

경로 설정: 코드 상단의 FINETUNED_LLM_PATH, BASE_MODEL_NAME_OR_PATH, VECTOR_DB_PATH, WIKI_DATA_PATH, ADVANCED_PROMPTS_PATH 를 실제 환경에 맞게 정확히 수정하세요. 특히 파인튜닝된 어댑터 경로와 베이스 모델 이름이 중요합니다.
라이브러리 설치: 필요한 모든 라이브러리가 설치되어 있는지 확인하세요. 특히 peft 라이브러리는 qLoRA 모델 로딩에 필수적일 수 있습니다.
Bash

pip install langchain langchain-community langchain-huggingface chromadb sentence-transformers torch transformers accelerate bitsandbytes pypdf wikipedia peft # 필요한 라이브러리 설치
RAG 데이터 준비: WIKI_DATA_PATH와 ADVANCED_PROMPTS_PATH에 RAG에 사용할 실제 데이터 파일들을 위치시키세요. (위키 데이터는 예시 키워드 외에 프로젝트에 필요한 내용을 추가하거나, 직접 관련 텍스트 파일을 준비하는 것이 좋습니다.)
GPU 메모리: 파인튜닝된 LLM을 로드하고 실행하기에 충분한 GPU VRAM이 필요합니다. 메모리가 부족하면 로딩에 실패하거나 매우 느릴 수 있습니다. (필요시 BitsAndBytesConfig를 사용한 양자화 설정을 조정하세요.)
실행:

1. 위 코드를 conversational_rag.py 와 같은 이름으로 저장합니다.
2. 터미널에서 실행합니다: python conversational_rag.py
3. 실행되면 설정 정보와 함께 각 단계별 진행 상황이 출력됩니다.
4. [User Input (Korean)]: 프롬프트가 나타나면 간단한 한국어 프롬프트를 입력하고 엔터를 누릅니다.
5. 시스템이 RAG 검색 및 LLM 추론을 통해 상세한 영어 프롬프트를 생성하여 출력합니다.
6. 이전 대화를 기억하므로, 다음 턴에는 "거기에 파란색 모자를 추가해줘" 와 같이 이전 프롬프트를 수정/보완하는 지시를 내릴 수 있습니다.
7. exit를 입력하면 프로그램이 종료됩니다.
